In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

In [ ]:
import requests
import io
import os

def _csv_from_path_or_url(path, url):
    if os.path.isfile(path):
        return path
    else:
        r = requests.get(url)
        r.raise_for_status()
        # After fetching from the URL, cache at the path
        contents = r.text
        with open(path, "w") as f:
            f.write(contents)
        return path

In [ ]:
csv = _csv_from_path_or_url(
    path="south-korea-active-cases.csv",
    url="https://qbin.io/bs-mixed-gv7a/raw",
)
df = pd.read_csv(csv)
(df.info(), df.head(), df.plot(y='active_cases'))

Try fitting "Gaussian" with form $f(x) = y_0 + a e^{-\frac{(x - m)^2}{b}}$:

In [ ]:
def gaussian(x, a, b, m, y0):
    return y0 + a * np.exp(-(x - m)**2 / b)

In [ ]:
pre_peak_data = df[df['days_since_feb_17'] < 24]
(fit_, cov) = curve_fit(
    gaussian,
    xdata=pre_peak_data['days_since_feb_17'],
    ydata=pre_peak_data['active_cases'],
    p0=[1000, 200, 20, 0],
)
print(f"f ~ y0 + a exp(-(x-m)^2 / b)")
print(f"a={fit_[0]:0.2f} b={fit_[1]:0.2f} m={fit_[2]:0.2f} y0={fit_[3]:0.2f}")
df['gaussian_estimate'] = [gaussian(x, *fit_) for x in df.index]
ax = df.plot(y=['active_cases', 'gaussian_estimate'])